In [7]:
import os
import glob
import numpy as np
import nibabel as nib
import pydicom
from pathlib import Path
from tqdm import tqdm

def get_direction_label(vec):
    """Convert orientation vector to direction label"""
    abs_vec = np.abs(vec)
    dominant_idx = np.argmax(abs_vec)
    dominant_val = vec[dominant_idx]
    
    if dominant_idx == 0:
        return 'RL' if dominant_val > 0 else 'LR'
    elif dominant_idx == 1:
        return 'AP' if dominant_val > 0 else 'PA'
    else:
        return 'FH' if dominant_val > 0 else 'HF'

def determine_dicom_orientation(iop):
    """Determine row and column directions from ImageOrientationPatient"""
    row_vec = np.array(iop[:3])
    col_vec = np.array(iop[3:6])
    
    dim2_dir = get_direction_label(row_vec)
    dim1_dir = get_direction_label(col_vec)
    
    dim2_axis = 0 if dim2_dir in ['LR', 'RL'] else (1 if dim2_dir in ['AP', 'PA'] else 2)
    dim1_axis = 0 if dim1_dir in ['LR', 'RL'] else (1 if dim1_dir in ['AP', 'PA'] else 2)
    
    used_axes = {dim2_axis, dim1_axis}
    dim0_axis = (set([0, 1, 2]) - used_axes).pop()
    dim0_dir = ['RL', 'AP', 'FH'][dim0_axis]
    
    return [dim0_dir, dim1_dir, dim2_dir]

def get_orientation_from_dicom(series_path):
    """Get orientation from DICOM"""
    dcm_files = glob.glob(os.path.join(series_path, '*.dcm'))
    
    if len(dcm_files) == 0:
        return None, "No DICOM files found"
    
    if len(dcm_files) == 1:
        return None, "Multi-frame DICOM (skipped)"
    
    slices = [pydicom.dcmread(f) for f in dcm_files]
    orientation = determine_dicom_orientation(slices[0].ImageOrientationPatient)
    
    return orientation, None

def create_affine_from_orientation(orientation):
    """
    Create affine matrix from orientation codes
    
    Args:
        orientation: List like ['FH', 'AP', 'LR'] representing [dim0, dim1, dim2]
    
    Returns:
        4x4 affine matrix
    """
    # Get orientation string (last letter of each direction)
    orient_str = ''.join([d[-1] for d in orientation])
    
    # Map direction codes to axis and sign
    # L/R = x-axis, P/A = y-axis, H/F = z-axis
    axis_map = {'L': 0, 'R': 0, 'P': 1, 'A': 1, 'H': 2, 'F': 2}
    sign_map = {'L': -1, 'R': 1, 'P': -1, 'A': 1, 'H': -1, 'F': 1}
    
    affine = np.zeros((4, 4))
    
    # Fill in the rotation part (unit spacing)
    for i, orient_code in enumerate(orient_str):
        axis = axis_map[orient_code]
        sign = sign_map[orient_code]
        affine[axis, i] = sign * 1.0  # unit spacing
    
    # Set the homogeneous coordinate
    affine[3, 3] = 1.0
    print(affine)
    assert False
    
    return affine

def reorient_to_lps(volume, mask, original_orientation):
    """
    Reorient volume and mask from original orientation to LPS (RAS+ in NiBabel)
    
    LPS means:
    - dim0: Right to Left (x-axis)
    - dim1: Anterior to Posterior (y-axis)
    - dim2: Inferior to Superior (z-axis) or Foot to Head
    
    Args:
        volume: numpy array of shape (d0, d1, d2)
        mask: numpy array of shape (d0, d1, d2)
        original_orientation: List like ['FH', 'AP', 'LR']
    
    Returns:
        reoriented_volume, reoriented_mask, transformation_info
    """
    target_orientation1 = ['RL', 'AP', 'IS']  # LPS / RAS+
    target_orientation2 = ['RL', 'AP', 'FH']  # LPS / RAS+
    
    # Map orientation codes to axis indices
    axis_map = {
        'LR': 0, 'RL': 0,  # x-axis
        'AP': 1, 'PA': 1,  # y-axis
        'FH': 2, 'HF': 2, 'IS': 2, 'SI': 2  # z-axis
    }
    
    # Determine axis permutation
    # For each target dimension, find which original dimension corresponds to it
    perm = []
    for target_code in target_orientation1:
        target_axis = axis_map[target_code]
        # Find which original dimension has the same anatomical axis
        for i, orig_code in enumerate(original_orientation):
            if axis_map[orig_code] == target_axis:
                perm.append(i)
                break
    
    # Transpose to correct axis order
    volume_reoriented = np.transpose(volume, perm)
    mask_reoriented = np.transpose(mask, perm)
    
    # Determine flipping for each dimension
    flips = []
    for i, target_code in enumerate(target_orientation1):
        orig_idx = perm[i]
        orig_code = original_orientation[orig_idx]
        
        # Check if we need to flip this dimension
        # We need to flip if the direction is opposite
        # print(orig_code, target_code)
        need_flip = (orig_code != target_orientation1[i]) and (orig_code != target_orientation2[i])
        flips.append(need_flip)
        
        if need_flip:
            print(f"Flipping dimension {i} ({orig_code} -> {target_code})")
            volume_reoriented = np.flip(volume_reoriented, axis=i)
            mask_reoriented = np.flip(mask_reoriented, axis=i)
    
    # Make contiguous arrays
    volume_reoriented = np.ascontiguousarray(volume_reoriented)
    mask_reoriented = np.ascontiguousarray(mask_reoriented)
    
    transform_info = {
        'original_orientation': original_orientation,
        'target_orientation': target_orientation1,
        'permutation': perm,
        'flips': flips
    }
    
    return volume_reoriented, mask_reoriented, transform_info

def convert_to_nnunet_format(volume_dir, mask_dir, series_base_path, output_base_dir):
    """
    Convert .npy volumes and masks to nnUNet format with .nii.gz files
    All data will be reoriented to LPS (RAS+) orientation
    
    Args:
        volume_dir: Directory containing volume .npy files
        mask_dir: Directory containing mask .npy files
        series_base_path: Path to original DICOM series folders
        output_base_dir: Base output directory (e.g., E:/Topcow/Dataset501_IAD)
    """
    
    # Create output directories
    images_dir = Path(output_base_dir) / "imagesTr"
    labels_dir = Path(output_base_dir) / "labelsTr"
    
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    # Get all mask files
    mask_files = sorted(list(Path(mask_dir).glob('*.npy')))
    
    print(f"Found {len(mask_files)} mask files to process\n")
    print(f"Target orientation: LPS (RAS+)")
    print(f"  Dim 0: Right → Left")
    print(f"  Dim 1: Anterior → Posterior")
    print(f"  Dim 2: Inferior → Superior (Foot → Head)\n")
    
    successful = 0
    skipped = 0
    case_index = 1
    orientation_counts = {}
    
    for mask_file in tqdm(mask_files, desc="Converting to nnUNet format"):
        series_uid = mask_file.stem
        print(f"\nProcessing {series_uid}...")
        volume_file = Path(volume_dir) / f"{series_uid}.npy"
        series_path = os.path.join(series_base_path, series_uid)
        
        # Check if corresponding volume exists
        if not volume_file.exists():
            print(f"\nSkipped {series_uid}: Volume file not found")
            skipped += 1
            continue
        
        # Check if DICOM series exists
        if not os.path.exists(series_path):
            print(f"\nSkipped {series_uid}: DICOM series folder not found")
            skipped += 1
            continue
        
        try:
            # Load volume and mask
            volume = np.load(volume_file).astype(np.float32) / 255.0  # Normalize to [0, 1]
            mask = np.load(mask_file)
            
            # Check shapes match
            if volume.shape != mask.shape:
                print(f"\nSkipped {series_uid}: Shape mismatch - Volume {volume.shape} vs Mask {mask.shape}")
                skipped += 1
                continue
            
            # Skip if no mask present (optional)
            if np.sum(mask) == 0:
                print(f"\nSkipped {series_uid}: No mask present")
                skipped += 1
                continue
            
            # Get orientation from DICOM
            orientation, error = get_orientation_from_dicom(series_path)
            if error:
                print(f"\nSkipped {series_uid}: {error}")
                skipped += 1
                continue
            
            # Track orientation statistics
            orient_key = '-'.join(orientation)
            orientation_counts[orient_key] = orientation_counts.get(orient_key, 0) + 1
            
            # Reorient to LPS
            volume_lps, mask_lps, transform_info = reorient_to_lps(volume, mask, orientation)
            
            # Create LPS affine matrix (identity with proper spacing)
            affine_lps = np.array([
                [1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 1.0]
            ])
            
            # Ensure correct data types
            volume_data = volume_lps.astype(np.float32)
            mask_data = mask_lps.astype(np.uint8)
            
            # Create NIfTI images with LPS orientation
            volume_nifti = nib.Nifti1Image(volume_data, affine_lps)
            mask_nifti = nib.Nifti1Image(mask_data, affine_lps)
            
            # Set data type in header explicitly
            volume_nifti.header.set_data_dtype(np.float32)
            mask_nifti.header.set_data_dtype(np.uint8)
            
            # Save files
            volume_filename = f"CASE_{case_index:04d}_0000.nii.gz"
            volume_path = images_dir / volume_filename
            nib.save(volume_nifti, volume_path)
            
            mask_filename = f"CASE_{case_index:04d}.nii.gz"
            mask_path = labels_dir / mask_filename
            nib.save(mask_nifti, mask_path)
            
            successful += 1
            case_index += 1
            
        except Exception as e:
            print(f"\nError processing {series_uid}: {e}")
            import traceback
            traceback.print_exc()
            skipped += 1
            continue
    
    # Summary
    print(f"\n{'='*80}")
    print("CONVERSION SUMMARY")
    print(f"{'='*80}")
    print(f"Successfully converted: {successful}")
    print(f"Skipped: {skipped}")
    print(f"Total processed: {len(mask_files)}")
    print(f"\n{'='*80}")
    print("ORIGINAL ORIENTATION DISTRIBUTION:")
    print(f"{'='*80}")
    for orient, count in sorted(orientation_counts.items(), key=lambda x: -x[1]):
        print(f"  {orient}: {count} cases")
    print(f"{'='*80}")
    print(f"\nAll data reoriented to: LPS (RAS+)")
    print(f"  Dim 0: Right → Left")
    print(f"  Dim 1: Anterior → Posterior")
    print(f"  Dim 2: Inferior → Superior\n")
    print(f"Output directories:")
    print(f"  Images: {images_dir}")
    print(f"  Labels: {labels_dir}")
    print(f"{'='*80}")
    
    return successful, skipped


if __name__ == "__main__":
    volume_dir = r"./volume_uint8_256"
    mask_dir = r"./mask_256"
    series_base_path = r"E:\data_old\series"
    output_base_dir = r"./Dataset501_IAD"
    
    # Run conversion
    convert_to_nnunet_format(volume_dir, mask_dir, series_base_path, output_base_dir)

Found 170 mask files to process

Target orientation: LPS (RAS+)
  Dim 0: Right → Left
  Dim 1: Anterior → Posterior
  Dim 2: Inferior → Superior (Foot → Head)



Converting to nnUNet format:   0%|          | 0/170 [00:00<?, ?it/s]


Processing 1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381...


Converting to nnUNet format:   1%|          | 1/170 [00:01<03:53,  1.38s/it]


Processing 1.2.826.0.1.3680043.8.498.10076056930521523789588901704956188485...


...


Processing 1.2.826.0.1.3680043.8.498.98133633346919790888527055899070500258...


Converting to nnUNet format:  99%|█████████▉| 169/170 [07:52<00:04,  4.01s/it]


Processing 1.2.826.0.1.3680043.8.498.98665804676684243068057801371997368733...


Converting to nnUNet format: 100%|██████████| 170/170 [07:58<00:00,  2.81s/it]


CONVERSION SUMMARY
Successfully converted: 170
Skipped: 0
Total processed: 170

ORIGINAL ORIENTATION DISTRIBUTION:
  FH-AP-RL: 160 cases
  AP-HF-RL: 6 cases
  RL-HF-AP: 4 cases

All data reoriented to: LPS (RAS+)
  Dim 0: Right → Left
  Dim 1: Anterior → Posterior
  Dim 2: Inferior → Superior

Output directories:
  Images: Dataset501_IAD\imagesTr
  Labels: Dataset501_IAD\labelsTr
